In [28]:
import nltk.stem
import re
import numpy as np
import os
import nltk
import pandas as pd
from src.SemEvalData import SemEvalData
from src.JigsawData import JigsawData
from src.preprocessing import preprocess_bayes
import joblib
import numpy as np
import matplotlib.pyplot as plt
import sklearn.datasets
import sklearn.feature_extraction.text
import sklearn.naive_bayes
import sklearn.metrics
import sklearn.model_selection
import sklearn.pipeline

In [13]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Julia\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\wordnet.zip.


True

In [14]:
#stemmer = nltk.stem.SnowballStemmer('english')
lemmer = nltk.stem.WordNetLemmatizer()

In [7]:
## load data
train_data_semeval = SemEvalData()
train_data_semeval.load_data("data/tsd_trial.csv")
train_df_preprocessed = train_data_semeval.preprocess()

moron
bigot
idiots
idiots
stupid
idiot
o try to turn salem into some kind of newstupidfuckingassholetopia this will drive up the price of housing and displace  people down here having lived in san clemente for a time myself i think youre dumb as shit to have moved up here the fuck
stupid
dont make ignorant statements
idiot
mexicans
rapists
drug dealers
psychopath
dumb
hypocrite
ridiculous
morons
hypocrisy
stupidity
stupid
idiot
r cho
stupid
nanakuli
ignorant
ridiculous
useless
illegal immigrants crime  social parasites and welfare refugees
duterte is right killing druggies
islamism and a fascist dictator
idiots
the sock puppet
idiot
put this pathetic rapists
let him rot with stale meals and a saggy cot
liberalism leads to stupidity
laziness
largassitis
idiocy
stupid blind
white
stupid
idiot
sexual
acts with animals
stupid is as stupid does
nuts
pathetic
drug addicts
prostitutes
trumpniks
ridiculous
moron
ridiculous
stupid
stupid
idiots
jack ass
hole
cheetoface sphincterlipped lying
stu

In [21]:
train_data = {
    'sentence':  train_df_preprocessed.sentences.sum(),
    'toxicity_sentence': train_df_preprocessed.toxicity_sentence.sum()
        }

train_df = pd.DataFrame (train_data, columns = ['sentence','toxicity_sentence'])

In [22]:

train_df['sentence'] = train_df.apply(lambda row: preprocess_bayes(row.sentence), axis=1)
print(train_df.head(5))

because hes a moron and a bigot
its not any more complicated than that
how about we stop protecting idiots and let nature add some bleach to the gene pool
we can always submit their names for the darwin awards
if people  were  smart they would  boycott this  inept  airline  but   they  are  not  smart  so   rogue  businesses  like  this  one   still thrive   taking the idiots  for  a ride
trump claimed that russia will never invade the ukraine when russia already has  how stupid can people be
as long as your willing to pay a lot more for products you buy then fine
but you better not be going to costco and walmart to buy stuff because its cheaper
if so we get to call you a hypocritical wanker
only an idiot would use and believe anything this this republican propaganda machine publishes wwwrealclearpoliticscom
thanks a lot douchebag
youre the reason the portland buttfuckers are getting pushed out and moving down south to the wilsonville area and beyond
because of you all the stupid fucki

no only people who have sold and not rebought or rebought but in a cheaper geographical market than they sold have benefitted
people that think they benefitted by taking out equity to reno or do other things with but have not sold or have sold and upgraded will not have benefitted as a portion of their equity will disappear over the next couple of years with rate increases and probably will not return for a very long time decade or so
everybody else who just live in their homes and didnt borrow anything against them as they skyrocketed in value how have they benefitted
what the fuck is with this civil comments horseshit
we dont have enough censorship from these communist fuckwits at adn
adn can fucking choke
the idiot should be fired i personally would not tolerate his behavior
trudeau is a complete idiot just like his old man
bunch of entitled whiners and beggarsbut once again our inherent politeness makes us doormats and chumps
pence managed the transition
there is no way that he cou

In [23]:
def stemming(text):
    # Stem words
    #data[i] = ' '.join([stemmer.stem(word) for word in data[i].split()])
    data = ' '.join([lemmer.lemmatize(word) for word in text.split()])
    return data


In [24]:
train_df['sentence'] = train_df.apply(lambda row: stemming(row.sentence), axis=1)

In [29]:
# Perform a grid search to find the best hyperparameters
def grid_search(train):
    # Create a pipeline
    clf_pipeline = sklearn.pipeline.Pipeline([
        ('v', sklearn.feature_extraction.text.CountVectorizer(strip_accents='ascii', stop_words='english')),
        ('t', sklearn.feature_extraction.text.TfidfTransformer()), 
        ('c', sklearn.naive_bayes.MultinomialNB(fit_prior=True, class_prior=None))
        ])
    # Set parameters (name in pipeline + name of parameter)
    parameters = { 
        'v__ngram_range': [(1, 1), (1, 2), (1, 3), (1, 4)], 
        'v__lowercase': (True, False), 
        't__use_idf': (True, False), 
        'c__alpha': (0.3, 0.6, 1.0) }
    # Create a grid search classifier
    gs_classifier = sklearn.model_selection.GridSearchCV(clf_pipeline, parameters, cv=5, iid=False, n_jobs=2, scoring='accuracy', verbose=1)
    
    # Start a search (Warning: takes a long time if the whole dataset is used)
    # Slice: (train.data[:4000], train.target[:4000])
    gs_classifier = gs_classifier.fit(train.data, train.target)
    # Print results
    print('---- Results ----')
    print('Best score: ' + str(gs_classifier.best_score_))
    for name in sorted(parameters.keys()):
        print('{0}: {1}'.format(name, gs_classifier.best_params_[name]))

In [44]:
# Train and evaluate a model
def train_and_evaluate(train):
    
    # Convert to bag of words
    count_vect = sklearn.feature_extraction.text.CountVectorizer(strip_accents='ascii', stop_words='english', lowercase=True, ngram_range=(1,1))
    X = count_vect.fit_transform(train['sentence'])
    # Convert from occurrences to frequencies
    # Occurrence count is a good start but there is an issue: longer documents will have higher average count values than shorter documents, even though they might talk about the same topics.
    # To avoid these potential discrepancies it suffices to divide the number of occurrences of each word in a document by the total number of words in the document: these new features are called tf for Term Frequencies.
    transformer = sklearn.feature_extraction.text.TfidfTransformer()
    X = transformer.fit_transform(X)
    # Create a model
    model = sklearn.naive_bayes.MultinomialNB(alpha=0.3, fit_prior=True, class_prior=None)
    # Train the model
    model.fit(X, train['toxicity_sentence'])
    # Save models
    joblib.dump(count_vect, 'vectorizer.jbl')
    joblib.dump(transformer, 'transformer.jbl')
    joblib.dump(model, 'model.jbl')
    # Evaluate on training data
    print('-- Training data --')
    predictions = model.predict(X)
    accuracy = sklearn.metrics.accuracy_score(train['toxicity_sentence'], predictions)
    print('Accuracy: {0:.2f}'.format(accuracy * 100.0))
    print('Classification Report:')
    print(sklearn.metrics.classification_report(train['toxicity_sentence'], predictions))
    print('')
    # Evaluate with 10-fold CV
    print('-- 10-fold CV --')
    predictions = sklearn.model_selection.cross_val_predict(model, X, train['toxicity_sentence'], cv=10)
    accuracy = sklearn.metrics.accuracy_score(train['toxicity_sentence'], predictions)
    print('Accuracy: {0:.2f}'.format(accuracy * 100.0))
    print('Classification Report:')
    print(sklearn.metrics.classification_report(train['toxicity_sentence'], predictions))


In [45]:
train_and_evaluate(train_df)

-- Training data --
Accuracy: 96.18
Classification Report:
              precision    recall  f1-score   support

         0.0       0.96      0.98      0.97      1230
         1.0       0.96      0.94      0.95       762

    accuracy                           0.96      1992
   macro avg       0.96      0.96      0.96      1992
weighted avg       0.96      0.96      0.96      1992


-- 10-fold CV --
Accuracy: 70.63
Classification Report:
              precision    recall  f1-score   support

         0.0       0.74      0.81      0.77      1230
         1.0       0.64      0.54      0.59       762

    accuracy                           0.71      1992
   macro avg       0.69      0.68      0.68      1992
weighted avg       0.70      0.71      0.70      1992

